In [1]:
import numpy as np
import pandas as pd
from rdkit import rdBase, Chem
from rdkit.Chem import AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from mordred import Calculator, descriptors
import mordred

In [2]:
df = pd.read_csv('./data/origin.csv',index_col=0)
df

,entry,R1-,organocatalyst,organocatalyst(mol%),temp(℃),time(h),yield,R2_C,R2_H,under_O2,under_air,solvent_CH3CN,solvent_MeOH,solvent_toluene
0,1,[*]C1=CC=CC=C1,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,90,24.0,81.0,0,1,1,0,0,0,1
1,2,[*]C1=CC=CC=C1,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,90,12.0,14.0,0,1,1,0,0,0,1
2,3,[*]C1=CC=CC=C1,OC1=CC(C)=CC=C1C(O)=O,5.0,90,24.0,74.0,0,1,1,0,0,0,1
3,4,[*]C1=CC=CC=C1,OC1=CC(C)=CC=C1C(O)=O,5.0,90,12.0,15.0,0,1,1,0,0,0,1
4,5,[*]C1=CC=CC=C1,OC1=CC(OC)=CC=C1C(O)=O,5.0,90,24.0,90.0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2i,[*]C1=CC(Br)=CC=C1,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,10.0,23,72.0,95.0,1,0,1,0,1,0,0
116,2j,[*]C1=CC=CC(OC)=C1,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,10.0,23,48.0,89.0,1,0,1,0,1,0,0
117,2k,[*]C1=CC(C)=C(C)C=C1,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,10.0,23,72.0,99.0,1,0,1,0,1,0,0
118,2m,[*]C1=CSC=C1,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,10.0,60,48.0,83.0,1,0,1,0,1,0,0


In [3]:
mols_r1 = list(Chem.MolFromSmiles(smile) for smile in df['R1-'].values)
mols_organocatalyst = list(Chem.MolFromSmiles(smile) for smile in df['organocatalyst'].values)

# Rdkit記述子に変換

In [4]:
def toFingerRdkit(mols,prefix):
    descriptor_names = [(descriptor_name[0] + '_' + prefix) for descriptor_name in Descriptors.descList]
    descriptor_calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)
    rdkit_descriptors_results = [descriptor_calculator.CalcDescriptors(mol) for mol in mols]
    df_rdkit = pd.DataFrame(rdkit_descriptors_results, columns=descriptor_names)
    df_rdkit = df_rdkit[df_rdkit.columns[~df_rdkit.isnull().any()]]
    return df_rdkit

In [5]:
df_rdkit_r1 = toFingerRdkit(mols_r1, 'r1')
df_rdkit_organocatalyst = toFingerRdkit(mols_organocatalyst, 'organocatalyst')

In [6]:
print(df_rdkit_r1.shape)
print(df_rdkit_organocatalyst.shape)

(120, 208)
(120, 208)


# Mordred記述子の変換を行う

In [7]:
def toFingerMordred(mols):
    mordered_calculator = Calculator(descriptors,ignore_3D=True)
    df_mordred = mordered_calculator.pandas(pd.Series(mols))
    
    for column in df_mordred.columns:
        if df_mordred[column].dtypes == object:
            df_mordred[column] = df_mordred[column].values.astype(np.float32)
    
    df_mordered = df_mordred[df_mordred.columns[~df_mordred.isnull().any()]]
    return df_mordered

In [8]:
df_mordred_r1 = toFingerMordred(mols_r1)
df_mordred_organocatalyst = toFingerMordred(mols_organocatalyst)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120/120 [00:29<00:00,  4.06it/s]


In [41]:
df_rdkit_r1.to_csv('./data/method4/rdkit_r1.csv')
df_rdkit_organocatalyst.to_csv('./data/method4/rdkit_organocatalyst.csv')
df_mordred_r1.to_csv('./data/method4/mordred_r1.csv')
df_mordred_organocatalyst.to_csv('./data/method4/mordred_organocatalyst.csv')

In [42]:
df_dropped = df.drop(columns=['entry', 'R1-', 'organocatalyst'])
r1_names = ['rdkit_r1','mordred_r1']
organo_names = ['rdkit_organo', 'mordred_organo']

for r1_name, df_r1 in zip(r1_names, [df_rdkit_r1,df_mordred_r1]):
    for organo_name, df_organocatalyst in zip(organo_names, [df_rdkit_organocatalyst, df_mordred_organocatalyst]):
        _df = pd.concat([df_dropped,df_r1, df_organocatalyst], axis=1)
        _df.to_csv(f'./data/method4/dataset_{r1_name}_{organo_name}.csv')


KeyError: "['entry', 'R1-', 'organocatalyst'] not found in axis"

# 特徴量選択
1. 分散0の説明変数を除去
2. 9割以上が同じ値になる記述子を削除
3. 0.95以上の説明変数の組みがある場合どちらかを除去
4. オートスケーリング

In [18]:
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

def selectFeature(df):
    select = VarianceThreshold()
    select_values = select.fit_transform(df.values)
    df_selected = df[df.columns[select.get_support()]]
    
    st_threshold = 0.9
    selected = []
    for column in df_selected:
        value_counts = df_selected[column].value_counts(sort=True)
        # 全体に対して何割同じ値があるか
        score = value_counts.values[0] / len(df_selected[column].values)
        if score < st_threshold:
            selected.append(column)
    df_selected = df_selected[selected]
    
    # 相関が0.95以上のカラムを除去
    df_selected = deleteHighCorrColumn(df_selected)
    
    return df_selected
        

def deleteHighCorrColumn(df):
    threshold = 0.95
    df_corr = df.corr()
    df_corr = abs(df_corr)
    columns = df_corr.columns

    # 対角線の値を0にする
    for i in range(0, len(columns)):
        df_corr.iloc[i, i] = 0

    while True:
        columns = df_corr.columns
        max_corr = 0.0
        query_column = None
        target_column = None

        df_max_column_value = df_corr.max()
        max_corr = df_max_column_value.max()
        query_column = df_max_column_value.idxmax()
        target_column = df_corr[query_column].idxmax()

        if max_corr < threshold:
            # しきい値を超えるものがなかったため終了
            break
        else:
            # しきい値を超えるものがあった場合
            delete_column = None
            saved_column = None

            # その他との相関の絶対値が大きい方を除去
            if sum(df_corr[query_column]) <= sum(df_corr[target_column]):
                delete_column = target_column
                saved_column = query_column
            else:
                delete_column = query_column
                saved_column = target_column

            # 除去すべき特徴を相関行列から消す（行、列）
            df_corr.drop([delete_column], axis=0, inplace=True)
            df_corr.drop([delete_column], axis=1, inplace=True)

    return df[df_corr.columns]


In [43]:
df1 = pd.read_csv('./data/method4/dataset_mordred_r1_mordred_organo.csv',index_col=0)
df2 = pd.read_csv('./data/method4/dataset_rdkit_r1_rdkit_organo.csv',index_col=0)
df3 = pd.read_csv('./data/method4/dataset_rdkit_r1_mordred_organo.csv',index_col=0)
df4 = pd.read_csv('./data/method4/dataset_mordred_r1_rdkit_organo.csv',index_col=0)

In [44]:
selectFeature(df1).to_csv('./data/method4/mordred_r1_mordred_organo.csv')
selectFeature(df2).to_csv('./data/method4/rdkit_r1_rdkit_organo.csv')
selectFeature(df3).to_csv('./data/method4/rdkit_r1_mordred_organo.csv')
selectFeature(df4).to_csv('./data/method4/mordred_r1_rdkit_organo.csv')

In [52]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler


def scale(df):
    # scaling
    X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['yield']), df['yield'], test_size=0.2, random_state=0)
    ss = StandardScaler()
    X_train_scaled = ss.fit_transform(X_train.values)
    X_test_scaled = ss.transform(X_test.values)
    return [X_train_scaled,X_test_scaled,y_train,y_test]

import optuna
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMRegressor
def objective(trial, x, t, cv):
    # 1. ハイパーパラメータごとに探索範囲を指定
    n_estimaters = trial.suggest_int('n_estimators', 1, 100)
    max_depth = trial.suggest_int('max_depth', 1, 10)
    num_leaves = trial.suggest_int('num_leaves', 2, 10)
    min_child_weight = trial.suggest_loguniform("min_child_weight", 0.1, 10)
    subsample = trial.suggest_uniform("subsample",0.55, 0.95)    
    colsample_bytree = trial.suggest_uniform("subsample",0.55, 0.95)    

    # 2. 学習に使用するアルゴリズムを指定
    estimator = LGBMRegressor(
        n_estimators=n_estimaters,
        max_depth=max_depth,
        num_leaves=num_leaves,
        min_child_weight=min_child_weight,
        subsample=subsample,
        colsample_bytree= colsample_bytree,
        random_state=43
    )

    # 3. 学習の実行、検証結果の表示
    print('Current_params : ', trial.params)
    r2 = cross_val_score(estimator, x, t, cv=cv, scoring="r2").mean()
    return r2

In [53]:
# study オブジェクトの作成（最大化）
study = optuna.create_study(direction='maximize')
# 10分割交差検証によるチューニング
cv = 10
[X_train_scaled,X_test_scaled,y_train,y_test] = scale(df1)
study.optimize(lambda trial: objective(trial, X_train_scaled, y_train, cv), n_trials=50)

[I 2022-12-05 08:29:25,170] A new study created in memory with name: no-name-0655b805-3dec-49d2-9aa6-ca3ca4ae4162


Current_params :  {'n_estimators': 13, 'max_depth': 6, 'num_leaves': 10, 'min_child_weight': 2.2223139003619967, 'subsample': 0.6161277965056239}


[I 2022-12-05 08:29:25,788] Trial 0 finished with value: -0.13268061833247513 and parameters: {'n_estimators': 13, 'max_depth': 6, 'num_leaves': 10, 'min_child_weight': 2.2223139003619967, 'subsample': 0.6161277965056239}. Best is trial 0 with value: -0.13268061833247513.


Current_params :  {'n_estimators': 76, 'max_depth': 1, 'num_leaves': 10, 'min_child_weight': 0.7259630007704551, 'subsample': 0.6268481590938403}


[I 2022-12-05 08:29:26,391] Trial 1 finished with value: -0.1314168738595139 and parameters: {'n_estimators': 76, 'max_depth': 1, 'num_leaves': 10, 'min_child_weight': 0.7259630007704551, 'subsample': 0.6268481590938403}. Best is trial 1 with value: -0.1314168738595139.


...

[I 2022-12-05 08:30:05,295] Trial 47 finished with value: -0.15720256916531472 and parameters: {'n_estimators': 10, 'max_depth': 2, 'num_leaves': 6, 'min_child_weight': 0.6852119488257786, 'subsample': 0.6440043860484134}. Best is trial 28 with value: -0.07416216404075347.


Current_params :  {'n_estimators': 39, 'max_depth': 9, 'num_leaves': 5, 'min_child_weight': 1.4480764130269856, 'subsample': 0.6883661959502225}


[I 2022-12-05 08:30:05,984] Trial 48 finished with value: -0.08436940367715481 and parameters: {'n_estimators': 39, 'max_depth': 9, 'num_leaves': 5, 'min_child_weight': 1.4480764130269856, 'subsample': 0.6883661959502225}. Best is trial 28 with value: -0.07416216404075347.


Current_params :  {'n_estimators': 44, 'max_depth': 7, 'num_leaves': 7, 'min_child_weight': 1.997991859135557, 'subsample': 0.7137121747521387}


[I 2022-12-05 08:30:06,738] Trial 49 finished with value: -0.09316500418623883 and parameters: {'n_estimators': 44, 'max_depth': 7, 'num_leaves': 7, 'min_child_weight': 1.997991859135557, 'subsample': 0.7137121747521387}. Best is trial 28 with value: -0.07416216404075347.


In [64]:
best_params = study.best_params
best_model = LGBMRegressor(**best_params)
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import r2_score

# 予測値を算出)
best_model.fit(X_train_scaled, y_train)
train_y_in_cv = cross_val_predict(best_model, X_train_scaled, y_train, cv=10)
predict_y = best_model.predict(X_test_scaled)

fig = plt.figure(figsize=(18.0, 5.0))
ax_cv.set_title("Q2")
ax_cv.set_xlabel('pred') 
ax_cv.set_ylabel('exp')  

ax_ext.set_title("External Validation")
ax_ext.set_xlabel('pred')
ax_ext.set_ylabel('exp') 

ax_cv.scatter(train_y_in_cv, y_train)
ax_ext.scatter(predict_y, y_test)


print('$R^{2}$=' + str(round(r2_score(y_train, train_y_in_cv), 3)))
print('$R^{2}$=' + str(round(r2_score(predict_y, y_test), 3)))
fig.show()

$R^{2}$=0.319
$R^{2}$=-0.818


/var/folders/12/6r_lrg9d6kx949nw6gcq9ksh0000gn/T/ipykernel_39492/2064959925.py:28: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


<Figure size 1800x500 with 0 Axes>